# Assignment for #05, 06, and 07

Downloading auxiliary classes:

In [22]:
%%bash
#wget --no-check-certificate -O "chap3lib.7z" "https://docs.google.com/uc?export=download&id=1SuDvAXGqR17finK7rO9xsqQh2Hey1VVQ"
#7z x chap3lib.7z

Importing auxiliary classes:

In [28]:
from chap3lib import *

## A1.

**Objective:**

The goal of this assignment is to define the `Exp2Alg` interface, which defines the operations for constructing expressions in the `Exp2` language. You will need to fill in the methods that currently raise `NotImplementedError()`.

**Background:**

The `Exp2` language is a small functional kernel whose syntax includes constants (integers and booleans), variables, unary and binary operations, conditional expressions, local bindings, function definitions (including recursive ones), and function applications. Each of these syntactic constructs corresponds to a method in the `Exp2Alg` interface.

Object Algebras provide a way to define language syntax and semantics separately. The `Exp2Alg` acts as an interface (algebra) that defines operations for each language construct of `Exp2`.

In [24]:
class Exp2Alg:
    def vint(self, v):
      raise NotImplementedError()
    def vbool(self, v):
      raise NotImplementedError()
    def var(self, x):
      raise NotImplementedError()
    def _not(self, e):
      raise NotImplementedError()
    def plus(self, e1, e2):
      raise NotImplementedError()
    def minus(self, e1, e2):
      raise NotImplementedError()
    def div(self, e1, e2):
      raise NotImplementedError()
    def mult(self, e1, e2):
      raise NotImplementedError()
    def _and(self, e1, e2):
      raise NotImplementedError()
    def _or(self, e1, e2):
      raise NotImplementedError()
    def eq(self, e1, e2):
      raise NotImplementedError()
    def le(self, e1, e2):
      raise NotImplementedError()
    def cond(self, e1, e2, e3):
      raise NotImplementedError()
    def let(self, x, e1, e2):
      raise NotImplementedError()
    def fun(self, x, e):
      raise NotImplementedError()
    #再帰処理
    def rec(self, f, x, e):
      raise NotImplementedError()
    #関数の適用を定義
    def app(self, e1, e2):
      raise NotImplementedError()

## A2.

**Objective:**

The goal of this assignment is to practice representing `Exp2` language constructs using the Object Algebra pattern. You will define a Python function that takes an instance of an `Exp2Alg` implementation and uses its methods to construct the `Exp2` expression `let x = 10 in (x + 5)`.

**Background:**

As discussed in the lecture, `Exp2` expressions can be programmatically constructed in Python by calling methods on an object that implements the `Exp2Alg` interface. Each method on this `alg` object corresponds to a specific syntactic construct in the `Exp2` language.

For example:
* A constant integer like `10` would be created using `alg.cint(10)`.
* A variable like `x` would be created using `alg.var("x")`.
* An addition expression like `e1 + e2` would be created using `alg.plus(e1, e2)`.
* A let-binding expression `let x = e1 in e2` would be created using `alg.let("x", e1, e2)`.

**Task:**

Define a Python function, let's call it `let_example`, that takes a single argument:
* `alg`: An instance of a class that implements the `Exp2Alg` interface.

This function should use the methods of the `alg` object to construct and return the representation of the following `Exp2` expression:

`let x = 10 in (x + 5)`

In [25]:
def let_example(alg):
  return alg.let('x',alg.cint(10),alg.plus(alg.var('x'),alg.cint(5)))

## A3.

**Objective:**

The primary goal of this assignment is to implement a call-by-value evaluation function for the `Exp2` language. You will achieve this by creating a Python class named `Exp2Eval` that implements the `Exp2Alg` interface. The methods in your `Exp2Eval` class will not directly return values, but rather "computations" – functions that, when given an environment, will produce the evaluated result (a value or an error).

**Background:**

The call-by-value evaluation rules for `Exp2` expressions are specified formally as $[[e]]_{Env}$ on the lecture slides. A strategy for implementing such an evaluator using object algebras, where algebra methods return computations (functions), is also discussed on the slides. This is the strategy you must follow.

You will use parts of the provided `chap3lib.py` for this assignment, including:
* Value classes: `Value`, `VInt2`, `VBool2`, `Clos2`, and `ClosRec2`.
* Error classes: `Err2`, `E_Typ2` (type error), `E_Exec2` (execution error, e.g., division by zero), `E_Undef2` (unbound variable).
* Environment lookup function: `lookup_env(env, name)`.

In [26]:
class Exp2Eval(Exp2Alg):
  def cint(self,v):
    def _eval(env):
      return VInt2(v)
    return _eval
  def cbool(self,v):
    def _eval(env):
      return VBool2(v)
    return _eval
  def var(self, x):
    def _eval(env):
      val= lookup_env(env,x)
      if val is None: raise ErrU()
      return val
    return _eval
#---UnaryOperators--
  def _not(self,e):
    def _eval(env):
      v= e(env) #Evaluatesubexpression
      if not isinstance(v,CBool2):
        raise ErrT()
      return CBool2(not v.v)
    return _eval
  def plus(self,e1, e2):
    def _eval(env):
      v1= e1(env)
      v2= e2(env)
      if not isinstance(v1,CInt2) or not isinstance(v2,CInt2):
        raise ErrT()
      return CInt2(v1.v + v2.v)
    return _eval
  def minus(self,e1,e2):
    def _eval(env):
      v1= e1(env)
      v2= e2(env)
      if not isinstance(v1,CInt2) or not isinstance(v2,CInt2):
        raise ErrT()
      return CInt2(v1.v-v2.v)
    return _eval
  def let(self,x,e1,e2):
    def _eval(env):
      v1= e1(env)
      new_env= [(x,v1)] +env
      return e2(new_env)
    return _eval
  def fun(self,x,e):
    def _eval(env):
      return Clos2(x,e,env)#Capturecurrentenv
    return _eval
  def rec(self,f,x, e):
    def _eval(env):
      return ClosRec2(f,x,e,env)#Capturecurrentenv
    return _eval
  def app(self,e1,e2):
    def _eval(env):
      v_fun= e1(env)
      v_arg= e2(env)

      if isinstance(v_fun,Clos2):
        app_env= [(v_fun.x,v_arg)] +v_fun.env
        return v_fun.e(app_env)
      elif isinstance(v_fun,ClosRec2):
        app_env= [(v_fun.f,v_fun),(v_fun.x,v_arg)] +v_fun.env
        return v_fun.e(app_env)
      else:
        raise ErrT()
    return _eval

## A4.

**Objective:**

The goal of this assignment is to implement a derivation tree generator for the `Exp2` language under Call-by-Value operational semantics. You will create a Python class `Exp2EvalTreeGenerator` that implements the `Exp2Alg` interface. Instead of directly evaluating to a final value, each method in this algebra will contribute to constructing a representation of the expression's Abstract Syntax Tree (AST) and a "computation" that, when executed with an environment, produces an `EvalStepNode`. This `EvalStepNode` (or one of its specific subclasses `F1` through `F20` from `chap3lib.py`) represents the root of the derivation (sub)tree for that part of the expression.

**Background:**

* The `Exp2` language syntax and its `Exp2Alg` interface are defined in the lecture slides.
* The Call-by-Value operational semantics rules (F1-F20) are described in the lecture slides.
* The provided `chap3lib.py` file contains:
    * AST node classes (`ANode`, `AConst`, `AVar`, `AUnOpExp`, `ABinOpExp`, `ACond`, `ALet`, `AFun`, `ARec`, `AApp`, and specific operator classes like `APlus`, `ANotOp`).
    * Value classes (`Val2`, `VInt2`, `VBool2`) and Error classes (`Err2`, `E_Typ2`, `E_Exec2`, `E_Undef2`).
    * Environment utilities (`lookup_env`).
    * Closure classes `Clos2` and `ClosRec2` which store an `ANode` for the body and a callable to get an `EvalStepNode`. These are suitable for this assignment.
    * `EvalStepNode` class and its subclasses `F1` through `F20`, which you **must use** to represent each step in the derivation tree. Each `Fk` class constructor typically requires the current environment, the AST node of the expression being evaluated (`expr_ast`), the resulting value/error, and any premise `EvalStepNode`s.

In [27]:
class Exp2EvalTreeGenerator(Exp2Alg):
  def vint(self,v):
    return

You can check your `Exp2EvalTreeGenerator` is correct or not by:
```python
tree = let_example(Exp2EvalTreeGenerator())
print(tree.pretty_print())
```
The above code will generate LaTeX code to render the derivation tree.

## A5.

**Objective:**

The goal of this assignment is to define and implement a Call-by-Name (CbN) evaluation function for the `Exp2` language. This evaluation function must be consistent with the Call-by-Name operational semantics discussed in the lecture slides (rules F'11-F'17). You will implement this evaluator as a Python class, say `CbNEval2Alg`, that is a subclass of `Exp2Alg`.

**Background:**

Call-by-Name evaluation differs significantly from Call-by-Value. Key principles include:
* **Delayed Evaluation:** Arguments to functions and expressions bound in `let` are not evaluated until their value is actually needed.
* **Thunks:** Instead of passing values, CbN passes "thunks" (or "frozen expressions"). A thunk encapsulates an unevaluated expression and the environment in which it was defined. You should define `Thunk` class to represent them as a subclass of `Value`.
* **Environment:** The evaluation environment will bind variables not only to direct values (like integers or booleans if they've been forced) but also to these thunks.

You should refer to the lecture slides:
* Call-by-Name operational semantics (focusing on rules F'11, F'13, F'14, F'15, F'16).
* The general strategy of algebra methods returning computations (functions from environment to result) is still applicable, but the nature of these computations and the values they handle will change for CbN.

You will use parts of the provided `chap3lib.py`:
* Value classes: `Val2` (as a base), `VInt2`, `VBool2`.
* Error classes: `Err2`, `E_Typ2`, `E_Exec2`, `E_Undef2`.